In [12]:
import numpy as np
import pandas as pd
from io import BytesIO,StringIO
from nanoHUB.application import Application
from nanoHUB.pipeline.geddes.data import get_default_s3_client


application = Application.get_instance()
salesforce = application.new_salesforce_engine()

s3_client = get_default_s3_client(application)

Obtained Salesforce access token ...... True


Obtained Salesforce access token ...... True


In [3]:
contacts_in_cluster_df = salesforce.query_data('Select Contact__c, Tool_Usage_Cluster__c from ContactToolClusterAssociation__c')
contacts_df = salesforce.query_data('Select Id, nanoHUB_user_ID__c from Contact')

self_study = []
clustered = []

sf_contact_ids = contacts_df['Id'].to_list()

unique_contacts_in_clusters = set(contacts_in_cluster_df['Contact__c'].to_list())
bool_search = np.in1d(np.array(sf_contact_ids),np.array(list(unique_contacts_in_clusters)))

clustered = np.where(bool_search)[0].tolist()
self_study = np.where(~bool_search)[0].tolist()

clustered_NH_ids = contacts_df['nanoHUB_user_ID__c'][clustered]
self_study_NH_ids = contacts_df['nanoHUB_user_ID__c'][self_study]


[Success] Bulk job creation successful. Job ID = 7505w00000b57O3AAI
{"id":"7505w00000b57O3AAI","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-16T12:59:41.000+0000","systemModstamp":"2021-11-16T12:59:42.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000b57O3AAI","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-16T12:59:41.000+0000","systemModstamp":"2021-11-16T12:59:47.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":98069,"retries":0,"totalProcessingTime":4517}
[Success] Bulk job completed successfully.
[Success] B

In [4]:
self_study_NH_df = self_study_NH_ids.to_frame()
display(self_study_NH_df)


,nanoHUB_user_ID__c
0,998.0
1,1683.0
2,1684.0
3,1685.0
4,1686.0
...,...
247745,81398.0
247746,337254.0
247747,62873.0
247748,105411.0


In [5]:
salesforce_query = '''
SELECT Id, nanoHUB_user_ID__c 
FROM Contact
    WHERE Id NOT IN (Select Contact__c from ContactToolClusterAssociation__c)
'''

In [6]:
result = salesforce.query_data(salesforce_query)
display(result)

[Success] Bulk job creation successful. Job ID = 7505w00000b57PfAAI
{"id":"7505w00000b57PfAAI","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-16T13:01:12.000+0000","systemModstamp":"2021-11-16T13:01:12.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000b57PfAAI","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-16T13:01:12.000+0000","systemModstamp":"2021-11-16T13:01:12.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000b57PfAAI","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-16T13:01:12.000+0000","systemModst

,Id,nanoHUB_user_ID__c
0,0035w000034IS8EAAW,158298.0
1,0035w000034IQUpAAO,155491.0
2,0035w000035jWJkAAM,287673.0
3,0035w000034JJn7AAG,28717.0
4,0035w000034I4dhAAC,80867.0
...,...,...
184268,0035w000034I4w9AAC,82105.0
184269,0035w000034JoNfAAK,113556.0
184270,0035w000034JMFlAAO,30987.0
184271,0035w000034Jz4UAAS,205903.0


In [7]:
display(result.nanoHUB_user_ID__c.nunique())

184255

In [8]:
intersection = pd.merge(result, self_study_NH_ids, how='inner', on=['nanoHUB_user_ID__c'])

In [9]:
display(intersection.nunique())

Id                    184273
nanoHUB_user_ID__c    184255
dtype: int64

In [10]:
difference1 = result.merge(
    self_study_NH_df, on='nanoHUB_user_ID__c', how='outer', suffixes=['', '_'], indicator=True
).loc[lambda x : x['_merge']!='both']

display(difference1)

,Id,nanoHUB_user_ID__c,_merge


In [4]:
from nanoHUB.repositories import UnclassifiedUsers, ClassroomUsers

In [16]:
repo = UnclassifiedUsers(salesforce)
unclassified_users_df = repo.get_all()
display(unclassified_users_df)

[Success] Bulk job creation successful. Job ID = 7505w00000b57QdAAI
{"id":"7505w00000b57QdAAI","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-16T13:03:39.000+0000","systemModstamp":"2021-11-16T13:03:39.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000b57QdAAI","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-16T13:03:39.000+0000","systemModstamp":"2021-11-16T13:03:50.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":201378,"retries":0,"totalProcessingTime":10641}
[Success] Bulk job completed successfully.


,Id,nanoHUB_user_ID__c
0,0035w000034IS8EAAW,158298.0
1,0035w000034IQUpAAO,155491.0
2,0035w000035jWJkAAM,287673.0
3,0035w000034JJn7AAG,28717.0
4,0035w000034I4dhAAC,80867.0
...,...,...
184268,0035w000034I4w9AAC,82105.0
184269,0035w000034JoNfAAK,113556.0
184270,0035w000034JMFlAAO,30987.0
184271,0035w000034Jz4UAAS,205903.0


In [17]:
full_path = 'unclassified_users.parquet.gzip' 
bucket_name = 'nanohub.processed'



_buf = BytesIO()
unclassified_users_df.to_parquet(_buf, index=False)
_buf.seek(0)
s3_client.put_object(Bucket=bucket_name, Body=_buf.getvalue(), Key=full_path)

{'ResponseMetadata': {'RequestId': 'tx00000000000000002ce25-006193ac4a-a6d52f-my-store',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 16 Nov 2021 13:04:10 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'etag': '"89c00c100ac8a1c6b37c076348ce25ff"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx00000000000000002ce25-006193ac4a-a6d52f-my-store',
   'strict-transport-security': 'max-age=15724800; includeSubDomains'},
  'RetryAttempts': 0},
 'ETag': '"89c00c100ac8a1c6b37c076348ce25ff"'}

In [20]:
repo = ClassroomUsers(salesforce)
classroom_users_df = repo.get_all()
display(classified_users_df)

full_path = 'classroom_users.parquet.gzip' 
bucket_name = 'nanohub.processed'

s3_client = get_default_s3_client(application)

_buf = BytesIO()
classroom_users_df.to_parquet(_buf, index=False)
_buf.seek(0)
s3_client.put_object(Bucket=bucket_name, Body=_buf.getvalue(), Key=full_path)

[Success] Bulk job creation successful. Job ID = 7505w00000b57QxAAI
{"id":"7505w00000b57QxAAI","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-16T13:05:13.000+0000","systemModstamp":"2021-11-16T13:05:14.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":296}
{"id":"7505w00000b57QxAAI","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-16T13:05:13.000+0000","systemModstamp":"2021-11-16T13:05:18.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":63477,"retries":0,"totalProcessingTime":3945}
[Success] Bulk job completed successfully.


,Id,nanoHUB_user_ID__c
0,0035w000034Jo08AAC,125212.0
1,0035w000034IrzvAAC,276235.0
2,0035w000034Is2GAAS,276281.0
3,0035w000034IXGEAA4,176759.0
4,0035w000034JtxGAAS,98366.0
...,...,...
63472,0035w000034IWuSAAW,166199.0
63473,0035w000034IWuOAAW,166195.0
63474,0035w000034IWuKAAW,166188.0
63475,0035w000034IX2eAAG,166411.0


{'ResponseMetadata': {'RequestId': 'tx00000000000000002ce39-006193acab-a6d52f-my-store',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 16 Nov 2021 13:05:47 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'etag': '"7f3ec9afee250a7dafab9394d8182ca0"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx00000000000000002ce39-006193acab-a6d52f-my-store',
   'strict-transport-security': 'max-age=15724800; includeSubDomains'},
  'RetryAttempts': 0},
 'ETag': '"7f3ec9afee250a7dafab9394d8182ca0"'}

In [17]:
for key in s3_client.list_objects(Bucket='nanohub.processed')['Contents']:
    obj = s3_client.get_object(Bucket='nanohub.processed', Key=key['Key'])
    print(key['Key'])
    if key['Key'].endswith('.parquet.gzip'):
        display(pd.read_parquet(BytesIO(obj['Body'].read())))
    else:
        display(pd.read_csv(BytesIO(obj['Body'].read())))
    

classroom_users.parquet.gzip


,Id,nanoHUB_user_ID__c
0,0035w000034Jo08AAC,125212.0
1,0035w000034IrzvAAC,276235.0
2,0035w000034Is2GAAS,276281.0
3,0035w000034IXGEAA4,176759.0
4,0035w000034JtxGAAS,98366.0
...,...,...
63472,0035w000034IWuSAAW,166199.0
63473,0035w000034IWuOAAW,166195.0
63474,0035w000034IWuKAAW,166188.0
63475,0035w000034IX2eAAG,166411.0


core_cost_clustered_users.csv


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,njohnson463,txu,ltong,svs470,eko,iezzoni21,gwm166,bmaganti,gpedrick,bsh319,...,sfedorikhina,dchun,asklar,cmpark,relwood,gthippeswamy,mayabhardwaj,jglass,noheneadu,dianemok
1,svpeders,clyons,bphester1,lbehrens,bluegreentwo,alimiadi,alentner12,mille273,mokomi89,mwjensen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ee21yin,zchng2,esunny2,chenchen,rbalik,tesler2,sramin3,nhamer2,bridgwater,mstephe2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,vmanane,mgsassenfeld,aelgkorv,jmmontes2,manaya,dmuse,danishr,champinhon,yvette,daesparza3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hkb,nizan123,haika,Tal,sduppleganger,shogun,guy23,itaikam,baraklit,hila,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,mhossai,ishsman,kjs1000,gu0,nihaoya,laplaceiitkgp,cchong,yoster84,ssuslov,pree2cool,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,torbentage,bjarkemalm,bluewolfrulz,jnunez,laur,robertpennington,nori86,marsmand,mhca,anderssig,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,kingya,sheng,silverjbj,pythais,benny7,spiderweb432,karlwr,jtli3,jazzphysicist,bumjin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,vkemper,ch_klar,alleinherrscher,thommy,masa,mkes1982,khalsa,mwidera,nilsb,zaradras,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,swetha443,mahi2234,nanotechnology530,jaejoon,ep4,mushtaq0455,subbu,spulli,praveen33,desidudes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


unclassified_users.parquet.gzip


,Id,nanoHUB_user_ID__c
0,0035w000034IS8EAAW,158298.0
1,0035w000034IQUpAAO,155491.0
2,0035w000035jWJkAAM,287673.0
3,0035w000034JJn7AAG,28717.0
4,0035w000034I4dhAAC,80867.0
...,...,...
184268,0035w000034I4w9AAC,82105.0
184269,0035w000034JoNfAAK,113556.0
184270,0035w000034JMFlAAO,30987.0
184271,0035w000034Jz4UAAS,205903.0
